In [13]:
import pandas as pd
from langchain.chains import LLMChain, ReduceDocumentsChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.chains import create_tagging_chain_pydantic
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.document_loaders import TextLoader
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain
from langchain.docstore.document import Document
from langchain.document_loaders.csv_loader import CSVLoader
import pandas as pd
from langchain.document_loaders import BigQueryLoader
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from google.cloud import bigquery
from pydantic import BaseModel

In [14]:
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access environment variables
api_key = os.getenv("OPENAI_KEY")

In [15]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=api_key)

In [16]:
PROJECT = "wagon-bootcamp-377120"
DATASET = "g_adventures_dataset"
TABLE = "one_month"

In [17]:
class UserTravelDetails(BaseModel):
    country: str
    max_budget: float
    min_budget: float
    departing_after: str
    departing_before: str
    max_duration: int
    min_duration: int

In [25]:
# Example usage:
user_travel_details = UserTravelDetails(
    country="Thailand",
    max_budget=1000,
    min_budget=0,
    departing_after="2024-04-01",
    departing_before="2024-04-15",
    max_duration=10,
    min_duration=5
)

In [19]:
def list_column_names():
    client = bigquery.Client(project=PROJECT)
    dataset_ref = client.dataset(DATASET)
    table_ref = dataset_ref.table(TABLE)

    # Get the table schema
    table = client.get_table(table_ref)

    # Extract and return the column names
    column_names = [field.name for field in table.schema]

    filtered_categories = [category for category in column_names if "Adult" in category and "Promotion Description" not in category]

    return filtered_categories

In [36]:
def list_all_column_names():
    client = bigquery.Client(project=PROJECT)
    dataset_ref = client.dataset(DATASET)
    table_ref = dataset_ref.table(TABLE)

    # Get the table schema
    table = client.get_table(table_ref)

    # Extract and return the column names
    column_names = [field.name for field in table.schema]

    return column_names

In [39]:
room_categories = list_column_names()
all_columns = list_all_column_names()

In [50]:
found_itineraries = ['Morocco: Deserts & Beaches']

In [54]:
def generate_sql_query(room_categories, user_travel_details, all_columns, found_itineraries):
    
    all_columns = list_all_column_names()
    all_columns_str = ",\n".join(all_columns)
    all_columns_str = all_columns_str + ","
    
    filtered_categories = [category for category in column_names if "Adult" in category and "Promotion Description" not in category]
    
    category_cases = []

    for category in room_categories:
        category_case = f"CASE WHEN {category} > 0 THEN {category} ELSE 1000000 END"
        category_cases.append(category_case)

    category_cases_str = ",\n".join(category_cases)

    query = f"""
    SELECT
        MAX(tour_operator) AS tour_operator,
        tour_name,
        itinerary_name,
        MAX(visited_countries) AS visited_countries,
        MAX(currency) AS currency,
        ARRAY_AGG(DISTINCT cost) AS Costs,
        MAX(duration) AS duration,
        ARRAY_AGG(CAST(start_date AS STRING) ORDER BY start_date) AS start_dates,
        MAX(Travel_Style) AS Travel_Style,
        MAX(Service_Level) AS Service_Level,
        MAX(Physical_Grading) AS Physical_Grading,
        MAX(Merchandising) AS Merchandising,
        MAX(Trip_Type) AS Trip_Type,
        MAX(itinerary) AS itinerary,
        MAX(url) AS url
FROM (
    SELECT
        {all_columns_str}
        LEAST(
            {category_cases_str}
        ) AS cost
    FROM {PROJECT}.{DATASET}.{TABLE}
) AS subquery
    WHERE 1 = 1
    """

    
    # Iterate through the provided filter criteria and add them to the query
    if user_travel_details.max_budget:
        query += f" AND cost <= {user_travel_details.max_budget}"

    if user_travel_details.departing_after:
        query += f" AND start_date >= '{user_travel_details.departing_after}'"

    if user_travel_details.departing_before:
        query += f" AND start_date <= '{user_travel_details.departing_before}'"

    query += f"""AND tour_name = '{found_itineraries[0]}'
    GROUP BY
        tour_name, itinerary_name;"""
    
    return query

print(generate_sql_query(room_categories, user_travel_details, all_columns, found_itineraries))


    SELECT
        MAX(tour_operator) AS tour_operator,
        tour_name,
        itinerary_name,
        MAX(visited_countries) AS visited_countries,
        MAX(currency) AS currency,
        ARRAY_AGG(DISTINCT cost) AS Costs,
        MAX(duration) AS duration,
        ARRAY_AGG(CAST(start_date AS STRING) ORDER BY start_date) AS start_dates,
        MAX(Travel_Style) AS Travel_Style,
        MAX(Service_Level) AS Service_Level,
        MAX(Physical_Grading) AS Physical_Grading,
        MAX(Merchandising) AS Merchandising,
        MAX(Trip_Type) AS Trip_Type,
        MAX(itinerary) AS itinerary,
        MAX(url) AS url
FROM (
    SELECT
        tour_operator,
tour_name,
itinerary_name,
visited_countries,
start_date,
duration,
url,
itinerary,
Travel_Style,
Service_Level,
Physical_Grading,
Merchandising,
Trip_Type,
currency,
Deluxe___Double___D___Adult,
Standard___Twin___D___Adult,
Standard___Adult,
En_suite_bathroom___Twin___D___Adult,
Shared_bathroom___Twin___D___Adult,
En_suite_bat

In [55]:
combined_query = generate_sql_query(room_categories, user_travel_details, all_columns, found_itineraries)

In [56]:
client = bigquery.Client(project="wagon-bootcamp-377120")
query_job = client.query(combined_query)
result = query_job.result()
df = result.to_dataframe()
df

,tour_operator,tour_name,itinerary_name,visited_countries,currency,Costs,duration,start_dates,Travel_Style,Service_Level,Physical_Grading,Merchandising,Trip_Type,itinerary,url
0,G Adventures,Morocco: Deserts & Beaches,None,Morocco,GBP,[849.0],11,"[2024-04-06, 2024-04-13]","18-to-Thirtysomethings - Fast, fresh, and fun ...",Basic - Simple and clean hotels and hostels; a...,2 - Light - Light walking and hiking suitable ...,None,Small Group - Group trips average 12 traveller...,"""Day 1: Arrive at any time. \n\nGet ready to c...",https://www.gadventures.com/trips/beach-tour-m...


In [16]:


# Define prompt
prompt_template = """Write a summary and only include the itinerary name, tour length, a summary of departure dates, summary of costs, travel style, physical grading, a summary of the itinerary and present the url to the user.
Include a mention of any potential interests: surfing.
"{text}"
Produce a summary paragraph not a list or bullet points.
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="text"
)


In [17]:
%%time
loader = BigQueryLoader(BASE_QUERY)
docs = loader.load()
itinerary_summary = stuff_chain.run(docs)

CPU times: user 117 ms, sys: 4.33 ms, total: 121 ms
Wall time: 43.1 s


In [18]:
print(itinerary_summary)

G Adventures offers the Peru Panorama tour, which has three different itineraries to choose from: Lares Trek, Cusco Stay, and Inca Trail. The tour lasts for 15 days and visits Peru. The departure dates for the tour are April 6, 8, 13, and 15 in 2024. The cost of the tour is £2769. The travel style is Classic, offering all the highlights and cultural experiences at a great price. The physical grading is 4, meaning it includes some high-altitude hikes and strenuous activities but is accessible to most healthy travelers. 

For the Lares Trek itinerary, the tour starts with a visit to Puno and Lake Titicaca, followed by a guided tour and homestay in a small village. The tour then continues to Cusco, where a full-day guided tour of the Sacred Valley is included. The highlight of this itinerary is the Lares Trek, a demanding hike that takes travelers through scenic landscapes and ancient ruins, culminating in a visit to Machu Picchu. The tour ends with a visit to Puerto Maldonado and the G L